In [1]:
import pandas as pd
import s3fs
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
import nltk.corpus

from sklearn.model_selection import train_test_split

# Data reading
The raw data are stored in the storage system S3 through SSP cloud.

In [2]:
key_id = '5LBXHMTZVFCOYVQRJRMN'
access_key = 'z0U21jRxrTyt3NLEtWCgh3euCCSgGCLBCRz9zOyT'
token = "eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiI1TEJYSE1UWlZGQ09ZVlFSSlJNTiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNjU1MDYyNTMzLCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imlkcmlzc2Eua29ua29ib0BlbnNhZS5mciIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJleHAiOjE2NTUxNDg5MzQsImZhbWlseV9uYW1lIjoiS09OS09CTyIsImdpdmVuX25hbWUiOiJJZHJpc3NhIiwiZ3JvdXBzIjpbXSwiaWF0IjoxNjU1MDYyNTM0LCJpc3MiOiJodHRwczovL2F1dGgubGFiLnNzcGNsb3VkLmZyL2F1dGgvcmVhbG1zL3NzcGNsb3VkIiwianRpIjoiZDQ5ODM2NTMtYjJhMy00NTlmLWJhZDktY2QyOWJlZWZlODJkIiwibG9jYWxlIjoiZW4iLCJuYW1lIjoiSWRyaXNzYSBLT05LT0JPIiwibm9uY2UiOiI2N2UyZjMzNC1lMzQ5LTRlYzAtOWY1Yy04NGVhZTA4ZjZkOWMiLCJwb2xpY3kiOiJzdHNvbmx5IiwicHJlZmVycmVkX3VzZXJuYW1lIjoiaWtvbmtvYm8iLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgZ3JvdXBzIGVtYWlsIiwic2Vzc2lvbl9zdGF0ZSI6ImE1MGYxZGQ1LThhNWItNGEzZS1iODg5LWM2MmQ5ZjNjMDM5OCIsInNpZCI6ImE1MGYxZGQ1LThhNWItNGEzZS1iODg5LWM2MmQ5ZjNjMDM5OCIsInN1YiI6IjBiMWM2YWM4LTcyZTQtNDkxYy1iMmYwLTIwOGZmYjhiMWNiNSIsInR5cCI6IkJlYXJlciJ9.e87ZqCZne_FpGavDqgH6D6jwJ8oCWBhvKX6TQ6-vQjnFVFpnAJZBBMFEj8BsjL5wLEr6PhPhxTl_y-vZ3-qCtQ"

fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'}, key = key_id, secret = access_key, token = token)

comms_df = pd.read_sas(fs.open("ikonkobo/commsdata/commsdata.sas7bdat"), format='sas7bdat')

In [8]:
comms_df["verbatims"]

0        b'Customer service is no longer an option with...
1        b'My phone was stollen 5 days ago  I got the n...
2        b'MTT needs to lower their plans    Im conside...
3        b'Phones were turned off due to past due bill ...
4                                          b'It was great'
                               ...                        
56552    b'Great customer service and very helpful  Tha...
56553    b'Acqueline fantastic great personality answer...
56554     b'americans serving americans always works best'
56555       b'Wish we had LTE or faster 4g in Olympia  WA'
56556    b'It was very frustrating when I was disconnec...
Name: verbatims, Length: 56557, dtype: object

In [ ]:
df_ = comms_df["verbatims"].to_frame()

In [ ]:
df_["verbatims"] = df_["verbatims"].apply(lambda x : x.decode("utf-8"))

In [ ]:
df_["verbatims_process"] = df_["verbatims"].apply(lambda x : re.sub("[\,.?!]", "", x))
df_["verbatims_process"] = df_["verbatims_process"].apply(lambda x : x.lower())

In [ ]:
df_

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(simple_preprocess(str(sentence), deacc=True))


def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df_.verbatims_process.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
#print(data_words[:1][0][:30])

In [ ]:
data_words

In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View

In [ ]:
id2word[1]

In [ ]:
from pprint import pprint
# number of topics
num_topics = 15
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
# pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
print(lda_model.get_topic_terms(5))
topic_word = [id2word[term[0]] for term in lda_model.get_topic_terms(5)]     
topic = '"'+','.join(topic_word)+'"'
topic

In [ ]:
def extract_topic(ldamodel, topic):
    topic_word = [id2word[term[0]] for term in ldamodel.get_topic_terms(topic)]     
    return '"'+','.join(topic_word)+'"'

list_topics = [extract_topic(lda_model, i) for i in range(10)]

In [ ]:
list_topics

In [ ]:
df__ = comms_df.select_dtypes(exclude='float64')
#df__
df__.drop(["verbatims", "resolution"], axis=1, inplace=True)
comms_df.drop(list(df__.columns), axis=1, inplace=True)
df_ = pd.get_dummies(df__, prefix_sep="_", drop_first=True)

In [ ]:
comms_df[df_.columns] = df_

In [ ]:
words = list(set([word for doc in comms_df["verbatims"].values for word in word_tokenize(doc.split())))

In [ ]:
df___ = comms_df["verbatims"].to_frame()
df___

In [ ]:
 for word in words:
        df___[word] = df___["verbatims"].apply(lambda x: x.count(word))

In [ ]:
comms_df["verbatims"].values[0].split()

In [ ]:
list(df__.columns)

In [ ]:
comms_df

# Data preprocessing

The project focuses on churn prediction and the dataset has 2 potential target variables which are 
+ *churn* : indicates wheteher customer churned 
+ *upsell_xsell* : indicates customer’s flag for cross-sell or up-sell. 
Since we focus on churn prediction, *upsell_xsell* will be deleted. 

Furthermore, the variables listed below are useless for predictive modeling and will be rejected :
+ city
+ city_lat
+ city_long
+ data_usage_amt
+ mou_onnet_6m_normal
+ mou_roam_6m_normal
+ region_lat
+ region_long
+ state_lat
+ state_long
+ tweedie_adjusted

In addition, we notice that character variables are showed in this format b'prime'. We will suppress characters 'b' and '.

In [ ]:
comms_df.drop(["upsell_xsell", "city", "city_lat", "city_long", "data_usage_amt", "mou_onnet_6m_normal", "mou_roam_6m_normal", "region_lat",
"region_long", "state_lat", "state_long", "tweedie_adjusted"], axis=1, inplace=True)

In [ ]:
list_vars_object = list(comms_df.select_dtypes(exclude = ['int64', 'float64']).columns)

for var in list_vars_object:
    comms_df[var] = comms_df[var].apply(lambda x : x.decode("utf-8"))

In [ ]:
df_val_mqtes = comms_df.isnull().sum().to_frame().reset_index()
df_val_mqtes.columns = ["variable", "nb_valeur_manquante"]
df_val_mqtes = df_val_mqtes[df_val_mqtes.nb_valeur_manquante > 0].reset_index(drop=True)
df_val_mqtes['pourcent_valeur_manquante'] = round(100 * df_val_mqtes['nb_valeur_manquante'] / comms_df.shape[0], 2)
df_val_mqtes = df_val_mqtes.sort_values('nb_valeur_manquante', ascending=False).reset_index(drop=True)


In [ ]:
list(df_val_mqtes.variable)

In [ ]:
l_ = [var for var in list(df_val_mqtes.variable) if len(comms_df[var].unique()) < 50]

l_

In [ ]:
comms_df["tot_drpd_pr1"].value_counts(ascending=False).to_frame().reset_index().iloc[0, 0]

In [ ]:
comms_df.values

In [ ]:
sns.histplot(comms_df[df_val_mqtes.variable[4]])#, bins=100)
plt.show()

# Data splitting

In [ ]:
comms_df.churn.describe()

The target variable churn seems to not have missing values. The dataset contains 12.13 % of customers churned. We will split the dataset to 70 % for training and 30% for test using *churn* as startify variable.

In [ ]:
X = comms_df.drop(columns=["churn"])
y = comms_df.churn

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)

In [ ]:
train = X_train 
train["churn"] = y_train

test = X_test 
test["churn"] = y_test